In [1]:
import pandas as pd
import glob
import os
import numpy as np

import tensorflow as tf
from tensorflow import keras

Read input files as a list.

In [2]:
df = pd.read_csv("cazy_df.csv",index_col=0)

In [3]:
df_names = df.columns[:-1]

In [4]:
d = pd.DataFrame( data={"classes" : df_names})

In [5]:
t = glob.glob("test_files/*.tsv") # replace this with your input files

In [6]:
classes = ["saprotroph","necrotroph","hemibiotroph","biotroph"]

Parse input files to a pandas dataframe where the rows correspond to each file and the columns to all annotations.

In [7]:
l=[]
for i in t:
    r = pd.read_csv(i,"\t")
    s = r[r["#ofTools"]==3].groupby("DIAMOND").sum()
    d1 = d.merge(s.reset_index(),how="left",left_on="classes",right_on="DIAMOND").iloc[:,[0,2]]
    d1.columns = ["classes",i.split("/")[1]]
    d1 = d1.transpose()
    d1.columns = d1.iloc[0]
    d1 = d1[1:]
    l.append(d1)

In [8]:
Xt= pd.concat(l,axis=0)

In [9]:
# Xt = Xt.transpose()
Xt = Xt.replace(np.nan,0).astype(int)

In [10]:
Xt

classes,AA0,AA1,AA1+CBM1,AA10,AA11,AA11+1.14.99.53,AA11+CBM18,AA12,AA12+AA8,AA13,AA13+CBM20,AA13+CBM20+1.14.99.55,AA15,AA15+CBM1,AA15+GH19,AA16,AA1_1,AA1_1+1.10.3.2,AA1_2,AA1_3,AA1_3+1.10.3.-,AA1_3+1.10.3.2,AA1_3+CBM18,AA1_3+CBM20,AA2,AA2+1.11.1.13,AA2+1.11.1.14,AA2+1.11.1.16,AA3,AA3_1,AA3_1+AA8,AA3_1+AA8+1.1.99.18,AA3_1+AA8+CBM1,AA3_1+AA8+CBM1+1.1.99.18,AA3_2,AA3_2+1.1.3.4,AA3_2+1.1.3.7,AA3_2+1.1.99.29,AA3_2+CBM1,AA3_3,...,GT73,GT76,GT8,GT8+2.4.1.186,GT90,GT91,PL1,PL11_1,PL11_2,PL14,PL14_3,PL14_4,PL14_5,PL15_1,PL1_10,PL1_2,PL1_3,PL1_4,PL1_4+4.2.2.10,PL1_7,PL1_7+4.2.2.2,PL1_8,PL1_9,PL20,PL20+4.2.2.14,PL26,PL27,PL33_2,PL35,PL3_2,PL3_2+4.2.2.2,PL4_1,PL4_1+4.2.2.23,PL4_3,PL4_3+4.2.2.23,PL4_5,PL7_4,PL9_1,PL9_2,PL9_3
Albugo_laibachii.ENA1_B.tsv,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
GCA_000002495.2.tsv,3,3,0,0,15,0,0,12,0,0,3,0,0,0,0,3,0,0,6,18,0,0,0,0,6,0,0,0,9,9,3,0,3,0,30,0,0,0,0,6,...,0,3,6,0,15,0,0,0,0,0,0,0,0,0,3,0,0,0,0,3,0,0,0,3,0,3,0,0,0,3,0,0,0,3,0,0,0,0,0,0
GCA_000002525.1.tsv,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,...,0,3,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Read model.

In [11]:
model = tf.keras.models.load_model('lspred_6_6_21.h5')

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 593)               352242    
_________________________________________________________________
dense_1 (Dense)              (None, 512)               304128    
_________________________________________________________________
dense_2 (Dense)              (None, 480)               246240    
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 1924      
Total params: 904,534
Trainable params: 904,534
Non-trainable params: 0
_________________________________________________________________


Run predict method on the model to get probabilities for all output classes.

In [13]:
pred = model.predict(Xt)

In [14]:
print(classes)
print(pred)

['saprotroph', 'necrotroph', 'hemibiotroph', 'biotroph']
[[1.4210328e-11 1.3124628e-06 1.1535585e-15 9.9999869e-01]
 [1.2182017e-16 8.3954603e-19 1.0000000e+00 6.5379519e-16]
 [1.0000000e+00 6.6392282e-19 5.5925851e-18 5.2694670e-13]]


Format probabilities in a dataframe.

In [15]:
p = pd.DataFrame(pred)
p.columns = classes
p.index = Xt.index
p

,saprotroph,necrotroph,hemibiotroph,biotroph
Albugo_laibachii.ENA1_B.tsv,1.421033e-11,1.312463e-06,1.153559e-15,9.999987e-01
GCA_000002495.2.tsv,1.218202e-16,8.395460e-19,1.000000e+00,6.537952e-16
GCA_000002525.1.tsv,1.000000e+00,6.639228e-19,5.592585e-18,5.269467e-13
